### 載入資料

為了後續增刪欄位以及圖表繪製的方便，我這裡採用 pandas 來載入資料。

**注意** 執行前請把 `jupyter notebook file` 放在與 `shot_logs.csv` **同層目錄**。

*PDF 與 Jupyter notebook 的內容大致相似*

In [65]:
import pandas as pd

data = pd.read_csv('shot_logs.csv', header=0)
data.head()

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,...,7.7,2,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,...,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,...,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,0,0,brian roberts,203148
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,...,17.2,2,missed,"Brown, Markel",203900,3.4,0,0,brian roberts,203148
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,...,3.7,2,missed,"Young, Thaddeus",201152,1.1,0,0,brian roberts,203148


### 資料前處理

因為要分析勝負，一定要有 TeamX vs Team Y 的資料，但是目前這些資訊在 MATCHUP 欄位被綁在一起了，必須先依照需求 split 開來。

In [66]:
# Define a function to parse the matchup string.

# FORMAT MAR 04, 2015 - CHA @ BKN
def parse_matchup(match, type):
    
    time, vs_info = match.split('-') # I do not select match-time as a feature.
    vs_info = vs_info.strip()
    
    # two case : TeamA v.s TeamB or TeamA @ TeamB
    if "vs." in vs_info:
        a,b = vs_info.split('vs.')
    else:
        a,b = vs_info.split('@')
    a = a.strip()
    b = b.strip()
    
    if type=="front":
        return a
    else:
        return b
    
def to_teamA(matchup):
    return parse_matchup(matchup,"front")
    
def to_teamB(matchup):
    return parse_matchup(matchup,"else")

In [67]:
# Parse matchup and add new columns into the dataset.

data['TEAMA'] = data['MATCHUP'].apply(to_teamA)
data['TEAMB'] = data['MATCHUP'].apply(to_teamB)

# Check it work or not.
data.head() # The new data is presented at the last two columns.

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id,TEAMA,TEAMB
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,...,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148,CHA,BKN
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,...,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148,CHA,BKN
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,...,missed,"Bogdanovic, Bojan",202711,0.9,0,0,brian roberts,203148,CHA,BKN
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,...,missed,"Brown, Markel",203900,3.4,0,0,brian roberts,203148,CHA,BKN
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,...,missed,"Young, Thaddeus",201152,1.1,0,0,brian roberts,203148,CHA,BKN


除了解析 matchup 外，我覺得 dataset 中有些資料並不會是影響勝負的關鍵，在這裡先將他們 drop 掉

In [93]:
data.drop("MATCHUP", axis=1, inplace=True) # Already used it.
data.drop("PERIOD", axis=1, inplace=True) # too scrappy.
data.drop("GAME_CLOCK", axis=1, inplace=True) # too scrappy.
data.drop("SHOT_CLOCK", axis=1, inplace=True) # too scrappy.
data.drop("CLOSEST_DEFENDER", axis=1, inplace=True) # we consider globally, not on specific case.
data.drop("CLOSEST_DEFENDER_PLAYER_ID", axis=1, inplace=True) # ID

# Check it work or not.
data.head()

ValueError: labels ['MATCHUP'] not contained in axis

因為我們考慮的是全局因素(是考慮某隊為什麼會贏，而不是 A 隊為什麼會贏 B 隊)，
所以我把特定的防守案例也就是 dataset 中的 CLOSEST_DEFENDER 欄位刪除了。

### 影響勝負的特徵推測

我認為勝負有兩個指標，一個是 `W` 表輸贏，另一個是 `FINAL_MARGIN` 表贏或輸的程度，這裡先採用 `FINAL_MARGIN` 做為主要的評估指標。

目前的 column 中，特別重要的應該會是每場比賽中，該球員的拿到球到射籃的總時長，出手次數以及得分率，如 SHOT_DIST、CLOSE_DEF_DIST 等都會反應於得分率中。所以接下來我要針對每場球隊的每場比賽中每個出場成員紀錄數據。

In [94]:
# Build a dataset for each team, each GAME and each player
team_info = data.groupby(['TEAMA','GAME_ID','player_name'])[['FINAL_MARGIN','SHOT_NUMBER']].max() # the last time
team_info[['TOUCH_TIME','FGM','PTS','DRIBBLES']] = data.groupby(['TEAMA','GAME_ID','player_name'])[['TOUCH_TIME','FGM','PTS','DRIBBLES']].sum()

# I add some new terms.
team_info['PLAYER_EFFICIENCY'] = team_info['PTS']/team_info['TOUCH_TIME']
#team_info['ATTACK_RATIO'] = team_info['TOUCH_TIME']/team_info['TOUCH_TIME'] + team_info['DRIBBLES']

team_info[['FINAL_MARGIN','PLAYER_EFFICIENCY']] # Let's have a look about this new term

FINAL_MARGIN  PLAYER_EFFICIENCY
TEAMA GAME_ID  player_name                                     
ATL   21400008 al horford                 -7           0.416667
               demarre carroll            -7           0.549451
               jeff teague                -7           0.314815
               kent bazemore              -7           0.294118
               kyle korver                -7           2.197802
               mike scott                 -7           1.600000
               paul millsap               -7           0.446927
               thabo sefolosha            -7           0.000000
      21400032 al horford                 10           1.019108
               demarre carroll            10           0.559441
               jeff teague                10           0.235479
               kent bazemore              10           1.428571
               kyle korver                10           0.930233
               mike scott                 10           3.750000
               paul millsap               10           0.683230
               pero antic                 10           0.769231
               thabo sefolosha            10           0.253165
      21400064 al horford                 -2           0.724638
               demarre carroll            -2           0.801887
               dennis schroder            -2           0.545455
               jeff teague                -2           0.058480
               kent bazemore              -2           0.117647
               kyle korver                -2           0.588235
               mike scott                 -2           1.904762
               paul millsap               -2           0.455840
               pero antic                 -2           0.909091
               thabo sefolosha            -2           0.000000
      21400071 al horford                 -3           0.880000
               demarre carroll            -3           1.195652
               dennis schroder            -3           0.204082
...                                      ...                ...
WAS   21400848 nene hilario              -20           0.558140
               otto porter               -20           1.023622
               ramon sessions            -20           0.119048
               rasual butler             -20           0.163934
      21400857 garrett temple             -8           0.000000
               john wall                  -8           0.091384
               kevin seraphin             -8           0.000000
               marcin gortat              -8           0.672269
               nene hilario               -8           0.335196
               otto porter                -8           0.548780
               ramon sessions             -8           0.154525
               rasual butler              -8           0.378378
      21400870 bradley beal                4           0.265957
               garrett temple              4           1.666667
               john wall                   4           0.182292
               kevin seraphin              4           0.000000
               marcin gortat               4           1.228070
               nene hilario                4           0.359551
               otto porter                 4           0.571429
               paul pierce                 4           0.588235
               ramon sessions              4           0.126582
      21400893 bradley beal               -5           0.177778
               john wall                  -5           0.199765
               kevin seraphin             -5           0.454545
               marcin gortat              -5           0.991736
               nene hilario               -5           0.000000
               otto porter                -5           1.454545
               paul pierce                -5           1.134752
               ramon sessions             -5           0.100000
               rasual butler              -5           3.000000


### 從個人表現來看

* PTS/TOUCH_TIME 可以得出 PLAYER_EFFICIENCY，即是讓這名球員碰球能獲得的效益有多大。

我認為這個指標能用於評量球員在優勢場面和劣勢場面的差異(被敵人限制住的程度，可以說是穩定性或抗壓性)，可用於球團的調度，比方有些球員於比分落後時 P_E 高於比分領先時，我想教練在調度時，或可考慮將這些球員在劣勢盤面派出場幫助團隊取分。

### 從全局評估

In [95]:
team_info = team_info.reset_index(['TEAMA','GAME_ID','player_name'])

In [96]:
# calculate the total static for each team.
team_global_stat = team_info.groupby('TEAMA')[['SHOT_NUMBER','FGM','PTS','TOUCH_TIME']].sum()

In [97]:
team_global_stat.head()

,SHOT_NUMBER,FGM,PTS,TOUCH_TIME
TEAMA,,,,
ATL,4546,2128,4822,12048.2
BKN,4373,1981,4319,13398.6
BOS,4381,1953,4294,9975.4
CHA,4669,2008,4301,13587.4
CHI,4602,2032,4447,12974.9


In [98]:
#team_global_stat['LIKE_TRIPLE'] = team_a_global_stat['FGM']/team_a_global_stat['PTS']
#team_global_stat # This feature make no sense. All of them are in the range [0.44-0.471], too close.
#team_global_stat.drop("LIKE_TRIPLE", axis=1, inplace=True)

### 追加對手資訊

In [99]:
# like we do for team A, but we change a -> b
team_info_b = data.groupby(['TEAMB','GAME_ID','player_name'])[['FINAL_MARGIN','SHOT_NUMBER']].max() # the last time
team_info_b[['TOUCH_TIME','FGM','PTS','DRIBBLES']] = data.groupby(['TEAMB','GAME_ID','player_name'])[['TOUCH_TIME','FGM','PTS','DRIBBLES']].sum()
team_info_b['PLAYER_EFFICIENCY'] = team_info_b['PTS']/team_info_b['TOUCH_TIME']

team_info_b[['FINAL_MARGIN','PLAYER_EFFICIENCY']] # Let's have a look about this new term
team_info_b.head()

FINAL_MARGIN  SHOT_NUMBER  TOUCH_TIME  FGM  \
TEAMB GAME_ID  player_name                                                     
ATL   21400008 amir johnson                  7           15        25.1    7   
               greivis vasquez               7           11        29.9    5   
               james johnson                 7            2         4.1    0   
               jonas valanciunas             7            5        12.4    4   
               kyle lowry                    7           11        38.6    3   

                                  PTS  DRIBBLES  PLAYER_EFFICIENCY  
TEAMB GAME_ID  player_name                                          
ATL   21400008 amir johnson        14        13           0.557769  
               greivis vasquez     12        22           0.401338  
               james johnson        0         2           0.000000  
               jonas valanciunas    8         5           0.645161  
               kyle lowry           6        39           0.155440

In [100]:
team_info_b = team_info_b.reset_index(['TEAMB','GAME_ID','player_name'])

In [101]:
# MERGE TEAM_A INFO AND TEAM_B INFO to team_global_stat

team_global_stat[['B_SHOT_NB','B_FGM','B_PTS','B_TOUCH_TIME']] = team_info_b.groupby('TEAMB')[['SHOT_NUMBER','FGM','PTS','TOUCH_TIME']].sum()
team_global_stat.head()

,SHOT_NUMBER,FGM,PTS,TOUCH_TIME,B_SHOT_NB,B_FGM,B_PTS,B_TOUCH_TIME
TEAMA,,,,,,,,
ATL,4546,2128,4822,12048.2,4205,1852,4140,10324.3
BKN,4373,1981,4319,13398.6,4103,1889,4207,10640.3
BOS,4381,1953,4294,9975.4,4392,1979,4344,12071.3
CHA,4669,2008,4301,13587.4,4156,1854,4092,11596.5
CHI,4602,2032,4447,12974.9,4514,2028,4373,13378.5


In [102]:
# ADD FINAL_MARGIN
team_global_stat['FINAL_MARGIN'] = team_global_stat['PTS'] - team_global_stat['B_PTS']

# ADD DIFF_TOUCH_TIME
team_global_stat['DIFF_TOUCH_TIME'] = team_global_stat['TOUCH_TIME'] - team_global_stat['B_TOUCH_TIME']

### DIFF_TOUCH_TIME

加上兩者 touch_time 差的目的是，想知道除了得分差外，哪一隊比較常控制住場面，總是能有攻擊時間

In [103]:
team_global_stat.head()

,SHOT_NUMBER,FGM,PTS,TOUCH_TIME,B_SHOT_NB,B_FGM,B_PTS,B_TOUCH_TIME,FINAL_MARGIN,DIFF_TOUCH_TIME
TEAMA,,,,,,,,,,
ATL,4546,2128,4822,12048.2,4205,1852,4140,10324.3,682,1723.9
BKN,4373,1981,4319,13398.6,4103,1889,4207,10640.3,112,2758.3
BOS,4381,1953,4294,9975.4,4392,1979,4344,12071.3,-50,-2095.9
CHA,4669,2008,4301,13587.4,4156,1854,4092,11596.5,209,1990.9
CHI,4602,2032,4447,12974.9,4514,2028,4373,13378.5,74,-403.6


### 計算每隊場數，數據平均

In [104]:
clear = data.groupby(['TEAMA','GAME_ID'])['W'].max().reset_index(['TEAMA','GAME_ID'])
get_win = clear[clear['W'] == 'W']

# Get the team list, and find which game they have winned.
teams = data['TEAMA'].unique().tolist()
game_played,game_won = [],[]
for i in teams:
    game_played.append(len(data[data.TEAMA == i]['GAME_ID'].unique()))
    game_won.append(len(get_win[get_win.TEAMA == i]))
    
team_global_stat['GAMES_PLAYED'] = game_played
team_global_stat['GAMES_WON'] = game_won

In [105]:
team_global_stat.head()

,SHOT_NUMBER,FGM,PTS,TOUCH_TIME,B_SHOT_NB,B_FGM,B_PTS,B_TOUCH_TIME,FINAL_MARGIN,DIFF_TOUCH_TIME,GAMES_PLAYED,GAMES_WON
TEAMA,,,,,,,,,,,,
ATL,4546,2128,4822,12048.2,4205,1852,4140,10324.3,682,1723.9,59,26
BKN,4373,1981,4319,13398.6,4103,1889,4207,10640.3,112,2758.3,59,13
BOS,4381,1953,4294,9975.4,4392,1979,4344,12071.3,-50,-2095.9,60,24
CHA,4669,2008,4301,13587.4,4156,1854,4092,11596.5,209,1990.9,61,34
CHI,4602,2032,4447,12974.9,4514,2028,4373,13378.5,74,-403.6,60,16


In [106]:
team_global_stat['SHOT_NORM'] = team_global_stat['SHOT_NUMBER']/team_global_stat['GAMES_PLAYED']
team_global_stat['B_SHOT_NORM'] = team_global_stat['B_SHOT_NB']/team_global_stat['GAMES_PLAYED']
team_global_stat['FGM_NORM'] = team_global_stat['FGM']/team_global_stat['GAMES_PLAYED']
team_global_stat['B_FGM_NORM'] = team_global_stat['B_FGM']/team_global_stat['GAMES_PLAYED']
team_global_stat['PTS_NORM'] = team_global_stat['PTS']/team_global_stat['GAMES_PLAYED']
team_global_stat['B_PTS_NORM'] = team_global_stat['B_PTS']/team_global_stat['GAMES_PLAYED']
team_global_stat['TOUCH_TIME_NORM'] = team_global_stat['TOUCH_TIME']/team_global_stat['GAMES_PLAYED']
team_global_stat['B_TOUCH_TIME_NORM'] = team_global_stat['B_TOUCH_TIME']/team_global_stat['GAMES_PLAYED']
team_global_stat['TOUCH_DIF'] = team_global_stat['TOUCH_TIME_NORM'] - team_global_stat['B_TOUCH_TIME_NORM']
team_global_stat['SHOT_VALUE'] = team_global_stat['PTS']/team_global_stat['SHOT_NUMBER']
team_global_stat['FINAL_MARGIN_NORM'] = team_global_stat['FINAL_MARGIN']/team_global_stat['GAMES_PLAYED']
team_global_stat['WIN_RATE'] = team_global_stat['GAMES_WON']/team_global_stat['GAMES_PLAYED']

norm_stat = pd.DataFrame(
                team_global_stat[
                    ['SHOT_NORM','B_SHOT_NORM','FGM_NORM','B_FGM_NORM','PTS_NORM','B_PTS_NORM',
                     'TOUCH_TIME_NORM','B_TOUCH_TIME_NORM','TOUCH_DIF','SHOT_VALUE','FINAL_MARGIN_NORM','WIN_RATE']
                   ]
            )

In [107]:
norm_stat

,SHOT_NORM,B_SHOT_NORM,FGM_NORM,B_FGM_NORM,PTS_NORM,B_PTS_NORM,TOUCH_TIME_NORM,B_TOUCH_TIME_NORM,TOUCH_DIF,SHOT_VALUE,FINAL_MARGIN_NORM,WIN_RATE
TEAMA,,,,,,,,,,,,
ATL,77.050847,71.271186,36.067797,31.389831,81.728814,70.169492,204.206780,174.988136,29.218644,1.060713,11.559322,0.440678
BKN,74.118644,69.542373,33.576271,32.016949,73.203390,71.305085,227.094915,180.344068,46.750847,0.987651,1.898305,0.220339
BOS,73.016667,73.200000,32.550000,32.983333,71.566667,72.400000,166.256667,201.188333,-34.931667,0.980142,-0.833333,0.400000
CHA,76.540984,68.131148,32.918033,30.393443,70.508197,67.081967,222.744262,190.106557,32.637705,0.921182,3.426230,0.557377
CHI,76.700000,75.233333,33.866667,33.800000,74.116667,72.883333,216.248333,222.975000,-6.726667,0.966319,1.233333,0.266667
CLE,66.542373,73.186441,31.016949,33.254237,69.237288,73.508475,235.532203,198.669492,36.862712,1.040499,-4.271186,0.796610
DAL,75.883333,73.916667,35.000000,33.350000,78.550000,74.466667,207.101667,200.475000,6.626667,1.035142,4.083333,0.800000
DEN,74.377049,70.278689,32.409836,31.950820,70.967213,70.032787,193.544262,202.757377,-9.213115,0.954155,0.934426,0.557377
DET,67.661017,71.406780,29.508475,33.033898,65.762712,72.864407,199.554237,181.052542,18.501695,0.971944,-7.101695,0.355932


In [ ]:
就結果來看， TOUCH_DIF 或 SHOT_VALUE 高於平均值的隊伍，有高機率能取得較大的場面差距，或是有較高機率能夠贏得比賽。